In [1]:
# Import
from __future__ import absolute_import, division, print_function

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)
tf.enable_eager_execution()

1.13.1


In [3]:
# Load Data 
filename = 'C:\\Users\\deniz\\Desktop\\Thesis of ML for AutoPas\\Data\\Batch3\\b3e1-6.txt'
arr = np.genfromtxt(filename, delimiter=',')
print(arr.shape)

(895, 30)


In [4]:
# Define how many parameters you have (it is assumed that the rest are classes)
parameter_count = 7

In [84]:
# Copy the array
data = arr

# Shuffle data and take 50% as test data
np.random.shuffle(data)
test_size = (np.ceil(data.shape[0] / 2)).astype(int)
train_params = data[test_size:,0:parameter_count]
train_labels = np.argmin(data[test_size:,parameter_count:], 1).astype(int)
test_params = data[:test_size,0:parameter_count]
test_labels = np.argmin(data[:test_size,parameter_count:], 1).astype(int)


# Normalize the data
def normalize01(array):
    divisor = np.max(array)
    for i in range(array.size):
        array[i] = array[i] / divisor

np.apply_along_axis(normalize01, 1, train_params)
np.apply_along_axis(normalize01, 1, test_params)
print(train_params[0])

[7.40740741e-06 3.70370370e-05 0.00000000e+00 0.00000000e+00
 3.70370370e-05 1.00000000e+00 1.00000000e-03]


In [85]:
# Check how the total data is distributed among the labels
dist = np.bincount(np.concatenate((train_labels, test_labels)))
print(dist)
print(np.argmax(dist))
print(np.max(dist) / data.shape[0])

[281  16  55   0  33   0   0  43   0   0   8   0   5   3   0   0   0   2
   0   0 409   0  40]
20
0.4569832402234637


In [86]:
# Model
model = keras.Sequential([
    #keras.layers.Dense(12, activation=tf.nn.relu),
    #keras.layers.Dense(17, activation=tf.nn.relu),
    #keras.layers.Dense(16, activation=tf.nn.relu),
    keras.layers.Dense(23, activation=tf.nn.softmax)
])

opt = keras.optimizers.Adam(lr=2)

model.compile(optimizer=opt, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [87]:
# Fun
model.fit(train_params, train_labels , epochs=50)

Epoch 1/50
447/447 [==============================] - 1s 1ms/sample - loss: 2.1493 - acc: 0.3960
Epoch 2/50
447/447 [==============================] - 0s 58us/sample - loss: 1.9208 - acc: 0.4944
Epoch 3/50
447/447 [==============================] - 0s 69us/sample - loss: 1.7124 - acc: 0.5324
Epoch 4/50
447/447 [==============================] - 0s 69us/sample - loss: 1.6827 - acc: 0.5391
Epoch 5/50
447/447 [==============================] - 0s 63us/sample - loss: 1.6528 - acc: 0.5414
Epoch 6/50
447/447 [==============================] - 0s 69us/sample - loss: 1.7403 - acc: 0.5101
Epoch 7/50
447/447 [==============================] - 0s 74us/sample - loss: 1.6815 - acc: 0.5481
Epoch 8/50
447/447 [==============================] - 0s 63us/sample - loss: 1.6130 - acc: 0.5638
Epoch 9/50
447/447 [==============================] - 0s 65us/sample - loss: 1.6755 - acc: 0.5436
Epoch 10/50
447/447 [==============================] - 0s 78us/sample - loss: 1.6350 - acc: 0.5414
Epoch 11/50
447/447 

In [88]:
predictions = model.predict(test_params)
test_loss, test_acc = model.evaluate(test_params, test_labels)
print('Test accuracy:', test_acc)

448/448 [==============================] - 0s 593us/sample - loss: 1.8546 - acc: 0.5469
Test accuracy: 0.546875


In [89]:
# Print prediction, result, and how certain the result is
best = np.argsort(predictions)
print(' Best  Guess1  Guess2  Certainty1      Certainty2')
for i in range(test_size):
    print(' ', test_labels[i], '\t', best[i][-1], '\t', best[i][-2], '\t', \
          "{:.2%}".format(predictions[i][best[i][-1]]), '\t', "{:.2%}".format(predictions[i][best[i][-2]]))

 Best  Guess1  Guess2  Certainty1      Certainty2
  20 	 20 	 0 	 55.05% 	 25.88%
  0 	 20 	 0 	 56.95% 	 42.42%
  0 	 20 	 0 	 59.72% 	 22.82%
  7 	 2 	 20 	 37.71% 	 30.30%
  0 	 0 	 20 	 73.80% 	 24.59%
  20 	 20 	 2 	 34.57% 	 32.91%
  20 	 20 	 0 	 54.92% 	 25.97%
  20 	 20 	 0 	 58.69% 	 40.66%
  20 	 0 	 20 	 70.55% 	 27.65%
  0 	 20 	 0 	 47.01% 	 20.70%
  20 	 20 	 0 	 60.27% 	 39.08%
  0 	 20 	 0 	 57.10% 	 42.26%
  20 	 20 	 0 	 55.24% 	 25.76%
  20 	 20 	 0 	 67.11% 	 32.22%
  20 	 20 	 0 	 60.54% 	 21.32%
  7 	 20 	 2 	 37.45% 	 27.91%
  0 	 0 	 20 	 74.67% 	 23.76%
  0 	 20 	 0 	 46.94% 	 20.75%
  0 	 20 	 0 	 59.97% 	 39.38%
  4 	 2 	 20 	 33.55% 	 33.01%
  0 	 20 	 0 	 58.15% 	 41.21%
  4 	 2 	 20 	 33.44% 	 33.06%
  2 	 2 	 20 	 33.58% 	 32.98%
  0 	 0 	 20 	 73.62% 	 24.76%
  20 	 20 	 2 	 35.62% 	 32.40%
  20 	 0 	 20 	 69.19% 	 28.92%
  0 	 20 	 0 	 61.85% 	 32.13%
  20 	 20 	 0 	 55.00% 	 25.92%
  20 	 20 	 0 	 63.34% 	 19.16%
  0 	 0 	 20 	 74.10% 	 24.30%
  7 	 2

  20 	 20 	 0 	 57.67% 	 24.13%
  20 	 20 	 0 	 61.77% 	 37.58%
  1 	 2 	 20 	 35.21% 	 32.03%
  0 	 20 	 0 	 57.51% 	 41.86%
  4 	 2 	 20 	 33.53% 	 33.02%
  0 	 20 	 0 	 60.73% 	 38.62%
  2 	 2 	 20 	 33.68% 	 32.95%
  0 	 20 	 0 	 56.95% 	 42.41%
  10 	 10 	 22 	 100.00% 	 0.00%
  20 	 20 	 2 	 33.53% 	 33.41%
  20 	 20 	 0 	 55.29% 	 25.72%
  20 	 20 	 0 	 80.58% 	 18.72%
  0 	 20 	 0 	 57.34% 	 42.03%
  20 	 20 	 2 	 33.59% 	 33.38%
  0 	 20 	 0 	 61.50% 	 32.47%
  7 	 2 	 20 	 37.11% 	 30.71%
  2 	 2 	 20 	 35.21% 	 32.04%
  20 	 20 	 0 	 55.29% 	 25.60%
  20 	 20 	 0 	 64.90% 	 34.44%
  12 	 2 	 20 	 37.12% 	 30.70%
  0 	 20 	 0 	 56.86% 	 42.51%
  20 	 20 	 0 	 55.65% 	 25.32%
  0 	 0 	 20 	 73.81% 	 24.58%
  12 	 2 	 20 	 33.52% 	 33.00%
  0 	 0 	 20 	 73.40% 	 24.96%
  0 	 20 	 0 	 55.87% 	 25.14%
  4 	 20 	 2 	 33.23% 	 33.00%
  0 	 0 	 20 	 73.80% 	 24.58%
  0 	 0 	 20 	 59.40% 	 38.08%
  0 	 20 	 0 	 57.27% 	 42.09%
  20 	 20 	 0 	 54.97% 	 25.94%
  0 	 0 	 20 	 70.57% 	 2

In [91]:
# Print general statistics about in how many guesses the AI would be correct
correct = np.zeros(best.shape[1])
most_occuring = np.sort(np.bincount(test_labels))[::-1]
for i in range(test_size):
    for j in range(correct.size):
        if best[i][-j-1] == test_labels[i]:
            correct[j] = correct[j] + 1
            break
np.set_printoptions(precision=3)
print('The count of guesses until correct choice:', correct.astype(int))
print('Cumilative chance that the choice was correct by:', \
      np.apply_along_axis(lambda x: x / test_size, 0, np.cumsum(correct))[0:5])
print('The count of most occuring tests:', most_occuring)
print('Cumilative chance that the choice was correct by:', \
      np.apply_along_axis(lambda x: x / test_size, 0, np.cumsum(most_occuring))[0:5])

The count of guesses until correct choice: [245 141  22   7   5   0   0  22   0   0   0   0   0   0   2   3   0   0
   1   0   0   0   0]
Cumilative chance that the choice was correct by: [0.547 0.862 0.911 0.926 0.938]
The count of most occuring tests: [198 144  30  22  21  14   8   5   3   2   1   0   0   0   0   0   0   0
   0   0   0   0   0]
Cumilative chance that the choice was correct by: [0.442 0.763 0.83  0.879 0.926]
